In [1]:
from datetime import *
from gmplot import gmplot
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime', 'time']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
#read dataset and take a look
stations = pd.read_csv("austin-bike/austin_bikeshare_stations.csv")
trips = pd.read_csv("austin-bike/austin_bikeshare_trips.csv")

trips[:5]

#transfer time into datetime
trips["start_time"] = trips["start_time"].map(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))

trips["hours"] = trips["start_time"].map(lambda x : x.hour)

In [4]:
end = datetime.datetime.strptime("2017", "%Y")
start = datetime.datetime.strptime("2014", "%Y")
trips_t_control = trips[(trips["start_time"] > start) & (trips["start_time"] <= end)]

In [5]:
def hours_to_category(hours):
    if hours >= 7 and hours <= 9:
        return "rush_m"
    elif hours >= 10 and hours <= 4:
        return "day time"
    elif hours >=5 and hours <= 7:
        return "rush_n"
    else:
        return "others"

In [6]:
trips_t_control["time_cat"] = trips_t_control["hours"].map(hours_to_category)


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
stations_close_McCombs = stations[(stations["latitude"] < 30.30) & (stations["latitude"] > 30.27) \
                                  & (stations["longitude"] < -97.6) & (stations["longitude"] > -97.8)\
                                 & (stations["status"] == "active")]
stations_close_McCombs

,latitude,location,longitude,name,station_id,status
0,30.27041,(30.27041 -97.75046),-97.75046,West & 6th St.,2537,active
9,30.28576,(30.28576 -97.74181),-97.74181,UT West Mall @ Guadalupe,2548,active
10,30.27336,(30.27336 -97.73805),-97.73805,State Capitol Visitors Garage @ San Jacinto & ...,2561,active
17,30.27466,(30.27466 -97.77028),-97.77028,MoPac Pedestrian Bridge @ Veterans Drive,3377,active
21,30.27217,(30.27217 -97.75246),-97.75246,Henderson & 9th,3685,active
31,30.27193,(30.27193 -97.73854),-97.73854,11th & San Jacinto,3291,active
38,30.27186,(30.27186 -97.73997),-97.73997,OFFICE/Main/Shop/Repair,1001,active
53,30.27260,(30.2726 -97.74127),-97.74127,Capitol Station / Congress & 11th,2497,active
56,30.28118,(30.28118 -97.75219),-97.75219,Pease Park,3464,active
57,30.28039,(30.28039 -97.73809),-97.73809,Bullock Museum @ Congress & MLK,2538,active


In [8]:
borrow_heatmap_df = trips_t_control[["start_station_id","time_cat"]].merge(stations_close_McCombs[["station_id", "latitude", "longitude","location"]],
                                                                           left_on = "start_station_id",
                                                                          right_on = "station_id")
borrow_heatmap_df[:5]

,start_station_id,time_cat,station_id,latitude,longitude,location
0,2538.0,others,2538,30.28039,-97.73809,(30.28039 -97.73809)
1,2538.0,others,2538,30.28039,-97.73809,(30.28039 -97.73809)
2,2538.0,others,2538,30.28039,-97.73809,(30.28039 -97.73809)
3,2538.0,others,2538,30.28039,-97.73809,(30.28039 -97.73809)
4,2538.0,others,2538,30.28039,-97.73809,(30.28039 -97.73809)


In [9]:
rush_m_heatmap_df = borrow_heatmap_df[borrow_heatmap_df["time_cat"] == "rush_m"]
rush_m_heatmap_weight = borrow_heatmap_df.groupby(["latitude", "longitude"])["location"].count()
rush_m_station_lat , rush_m_station_lon = zip(*rush_m_heatmap_weight.index.values)
#rush_m_heatmap_weight.values

rush_m_heatmap_weight_p = rush_m_heatmap_weight.values/ float(sum(rush_m_heatmap_weight.values))
rush_m_heatmap_weight_p

array([0.15154806, 0.03105054, 0.25002604, 0.05867912, 0.07602696,
       0.06152084, 0.09450553, 0.13605999, 0.00496928, 0.13561365])

In [10]:
gmap = gmplot.GoogleMapPlotter(30.28398,-97.73792, 13)


for i in range(0, len(rush_m_station_lat)):
    gmap.heatmap([rush_m_station_lat[i]], [rush_m_station_lon[i]], radius = rush_m_heatmap_weight_p[i] * 300)
# Draw
gmap.draw("rush_hour_in_morning.html")